## 第二题
$$
\begin{alignat}{2}
\min\quad & T& \\
\mbox{s.t.}\quad
&(1)\quad T\geq \sum_{i}\sum_jD_{ij}X_{ij}^k/V\\
&(2)\quad \sum_i\sum_kX_{ij}^k=\sum_l \sum_kX_{jl}^k\\
&(3)\quad \sum_i\sum_k X_{ij}^k=1\\
&(4)\quad \sum_j X_{0j}^k=1\\
&(5)\quad U_i^k+P_j\leq U_j^k+(1-X_{ij}^k)M \\
&(6)\quad \sum_iP_j X_{ij}^k\leq U_j^k\\
&(7)\quad U_i^k \leq Q
\end{alignat}
$$

$X_{ij}^k$: 0-1变量，第k辆车是否会从i到j地,南沙工厂用0表示

$T$: 耗时最多的那辆车花费的时间

$U_j^k$: 在j处装好货物后k车的总载重量

$P_i$: i地需要装载的货物量

$Q$: 货车的载重上限

$M$: 大M值

$V$: 车速

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import math

In [2]:
df=pd.read_excel('运营管理-距离.xlsx',sheet_name=['距离','货运量'])
df1=df['距离']
df2=df['货运量']

total_point=df1.columns.tolist()
plant_point=df2.index.tolist()

good_quantity={}
for plant in plant_point:
    good_quantity[plant]=df2.loc[plant,'amount']

    distance={}
for i in total_point:
    for j in total_point:
        distance[i,j]=df1.loc[i,j]
plant_s,distance=multidict(distance)
##############################
Q=5
V=60
vehicle=3
vehicle_list=list(range(1,vehicle+1))

In [3]:
m=Model('VRP')
X=m.addVars(total_point,total_point,vehicle_list,vtype=GRB.BINARY,name='X')
U=m.addVars(plant_point,vehicle_list,name='U')
T=m.addVar(name='T')

In [4]:
m.setObjective(T,GRB.MINIMIZE)
for k in vehicle_list:
    m.addConstr(quicksum(distance[i,j]*X[i,j,k]/V for i,j in plant_s)<=T)

for j in plant_point:
    m.addConstr(X.sum('*',j,"*")==X.sum(j,"*","*"))
    m.addConstr(X.sum('*',j,"*")==1)
for k in vehicle_list:
    m.addConstr(X.sum('南沙','*',k)==1)
for j in total_point:
    for k in vehicle_list:
        m.addConstr(X.sum("*",j,k)==X.sum(j,"*",k))

for i in plant_point:
    for j in plant_point:
        for k in vehicle_list:
            m.addConstr(U[i,k]+good_quantity[j]<=U[j,k]+(1-X[i,j,k])*5000)

for j in plant_point:
    for k in vehicle_list:
        m.addConstr(good_quantity[j]*X.sum("*",j,k)<=U[j,k])

for i in plant_point:
    for k in vehicle_list:
        m.addConstr(U[i,k]<=Q)

m.optimize()

Optimize a model with 101 rows, 88 columns and 558 nonzeros
Variable types: 13 continuous, 75 integer (75 binary)
Coefficient statistics:
  Matrix range     [7e-01, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Presolve removed 68 rows and 57 columns
Presolve time: 0.02s
Presolved: 33 rows, 31 columns, 135 nonzeros
Variable types: 6 continuous, 25 integer (24 binary)
Found heuristic solution: objective 4.3333333

Root relaxation: objective 2.333333e+00, 18 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0         4.33333    4.33333  0.00%     -    0s

Explored 0 nodes (47 simplex iterations) in 0.14 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 4.33333 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.333333333333e+00, best bound 

In [5]:
a1=pd.DataFrame(columns=total_point,index=total_point)
AA=m.getAttr('x',X)
for a in total_point:
    for b in total_point:
        a1.loc[a,b]=AA[a,b,3]
a1

,南沙,花都,高明,三水,珠海
南沙,0,1,0,0,0
花都,0,0,1,0,0
高明,1,0,0,0,0
三水,0,0,0,0,0
珠海,0,0,0,0,0


In [6]:
a2=pd.DataFrame(columns=total_point,index=total_point)
AA=m.getAttr('x',X)
for a in total_point:
    for b in total_point:
        a2.loc[a,b]=AA[a,b,2]
a2

,南沙,花都,高明,三水,珠海
南沙,0,0,0,1,0
花都,0,0,0,0,0
高明,0,0,0,0,0
三水,1,0,0,0,0
珠海,0,0,0,0,0


In [7]:
a1=pd.DataFrame(columns=total_point,index=total_point)
AA=m.getAttr('x',X)
for a in total_point:
    for b in total_point:
        a1.loc[a,b]=AA[a,b,1]
a1

,南沙,花都,高明,三水,珠海
南沙,0,0,0,0,1
花都,0,0,0,0,0
高明,0,0,0,0,0
三水,0,0,0,0,0
珠海,1,0,0,0,0


In [8]:
print('总时间是：'+str(m.objVal)+'小时')

总时间是：4.333333333333333小时


In [9]:
df1

,南沙,花都,高明,三水,珠海
南沙,0,95,78,43,80
花都,95,0,87,55,160
高明,78,87,0,45,140
三水,43,55,45,0,260
珠海,80,160,140,260,0


In [10]:
df2

,amount
花都,2
高明,3
三水,4
珠海,5
